- Author: Ben Du
- Date: 2020-11-30 00:15:05
- Title: IO in Rust
- Slug: io-in-rust
- Category: Computer Science
- Tags: Computer Science, programming
- Modified: 2021-11-22 16:27:01


 ** Things on this page are fragmentary and immature notes/thoughts of the author. Please read with your own judgement! **  

## Tips and Traps

1. Key strokes `CTRL` + `D` signals an EOF to stdin input.

2. Methods reading from stdin **appends** the input to the buffer rather than overwrite it!

3. Rust crates 
    [serde](https://crates.io/crates/serde)
    and
    [cron](https://crates.io/crates/ron)
    are popular serialization/deserialization libraries.

https://doc.rust-lang.org/std/io/trait.BufRead.html#method.lines
    
https://doc.rust-lang.org/rust-by-example/std_misc/file/read_lines.html
    
https://doc.rust-lang.org/std/io/struct.Stdin.html#method.read_line

In [2]:
use std::io::BufReader;
use std::fs::File;
use std::io::Read;
use std::io::Lines;

In [15]:
use std::io::{self, BufReader, BufWriter};
use std::io::prelude::*;
use std::fs::File;

## [std::fs::read_to_string](https://doc.rust-lang.org/std/fs/fn.read_to_string.html)

In [12]:
use std::fs;

In [17]:
fs::read_to_string("data.txt")

Ok("how\nare you\ndoing")

## [std::fs::read](https://doc.rust-lang.org/std/fs/fn.read.html)

Read the entire contents of a file into a bytes vector.
This is a convenience function for using `File::open` and `read_to_end` with fewer imports and without an intermediate variable. 
It pre-allocates a buffer based on the file size when available, 
so it is generally faster than reading into a vector created with `Vec::new()`.

In [19]:
fs::read("data.txt")

Ok([104, 111, 119, 10, 97, 114, 101, 32, 121, 111, 117, 10, 100, 111, 105, 110, 103])

## [std::fs::write](https://doc.rust-lang.org/std/fs/fn.write.html)

In [13]:
fs::write("o1.txt", "this is an example line output")

Ok(())

## std::fs::File

Read text from a file.

In [17]:
let f1 = File::open("data.txt");

In [18]:
f1

Ok(File { fd: 4, path: "/workdir/archives/blog/misc/content/2020/11/data.txt", read: true, write: false })

In [19]:
let mut f = File::open("data.txt")?;

In [20]:
let mut content = String::new();
f.read_to_string(&mut content)?;

In [21]:
content

"how\nare you\ndoing"

Write text into a file.

In [8]:
let mut file = File::create("output.txt").unwrap();
file

File { fd: 4, path: "/workdir/archives/blog/misc/content/2020/11/rust-io/output.txt", read: false, write: true }

In [9]:
file.write_all(b"Hello, world!");

## [std::io::LineWriter](https://doc.rust-lang.org/std/io/struct.LineWriter.html)

## std::io::BufReader

In [29]:
let br = BufReader::new(File::open("data.txt")?);
for line in br.lines() {
    println!("{:?}", line);
}

Ok("how")
Ok("")
Ok("are you")
Ok("doing")


()

In [31]:
let br = BufReader::new(File::open("data.txt")?);
for (idx, line) in br.lines().enumerate() {
    println!("Line {}: {:?}", idx, line);
}

Line 0: Ok("how")
Line 1: Ok("")
Line 2: Ok("are you")
Line 3: Ok("doing")


()

In [33]:
let br = BufReader::new(File::open("data.txt")?);
for (idx, line) in br.lines().enumerate() {
    println!("Line {}: {:?}", idx, line);
}

Line 0: Ok("how")
Line 1: Ok("")
Line 2: Ok("are you")
Line 3: Ok("doing")


()

## std::io::BufWriter

In [16]:
let f = File::create("o2.txt").expect("Unable to create file");
let mut bfw = BufWriter::new(f);
bfw.write_all(b"writing data\nusing std::io::BufWriter")

Ok(())

You have to call the method `BufWriter.flush` to force the buffer to output immediately.

In [19]:
bfw.flush()

Ok(())

Before a buffer is destructed, its content is output to the file.

In [18]:
{
    let f = File::create("o3.txt").expect("Unable to create file");
    let mut bfw = BufWriter::new(f);
    bfw.write_all(b"writing data\nusing std::io::BufWriter")
}

Ok(())

## Read From Stdin

Stdin::read_line reads a line (terminated by `\n`)
and returns a Result object. 
On success,
it returns `Ok(n)` where `n` is the number of bytes read from stdin.

In [2]:
let mut buffer = String::new();
let stdin = std::io::stdin();
let r = stdin.read_line(&mut buffer);

Error: cannot find value `pwd` in this scope

## Read/Write CSV in Rust 

Please refer to 
[Read and Write CSV Files in Rust](http://www.legendu.net/misc/blog/read-and-write-csv-files-in-rust/)
for details.

## Bundle File into Rust Application

Please refer to
[Bundle Resource Files into a Rust Application](http://www.legendu.net/misc/blog/bundle-resource-files-into-a-rust-application)
for more discussions.

## [fs::canonicalize](https://doc.rust-lang.org/std/fs/fn.canonicalize.html)

In [20]:
fs::canonicalize("./o1.txt")

Ok("/workdir/archives/blog/misc/content/2020/11/rust-io/o1.txt")

## [fs::copy](https://doc.rust-lang.org/std/fs/fn.copy.html)

In [21]:
fs::copy("o1.txt", "out1.txt")

Ok(30)

## [fs::create_dir](https://doc.rust-lang.org/std/fs/fn.create_dir.html)

In [22]:
fs::create_dir("my_dir")

Ok(())

## [fs::create_dir_all](https://doc.rust-lang.org/std/fs/fn.create_dir_all.html)

In [23]:
fs::create_dir_all("another/dir")

Ok(())

## [fs::metadata](https://doc.rust-lang.org/std/fs/fn.metadata.html)

In [24]:
fs::metadata("o1.txt")

Ok(Metadata { file_type: FileType(FileType { mode: 33188 }), is_dir: false, is_file: true, permissions: Permissions(FilePermissions { mode: 33188 }), modified: Ok(SystemTime { tv_sec: 1627506651, tv_nsec: 611396728 }), accessed: Ok(SystemTime { tv_sec: 1627506651, tv_nsec: 987468906 }), created: Err(Error { kind: Other, message: "creation time is not available for the filesystem" }), .. })

## [fs::read_dir](https://doc.rust-lang.org/std/fs/fn.read_dir.html)

In [25]:
fs::read_dir(".")

Ok(ReadDir("."))

In [27]:
for file in fs::read_dir(".").unwrap() {
    println!("{:?}", file);
}

Ok(DirEntry("./.ipynb_checkpoints"))
Ok(DirEntry("./another"))
Ok(DirEntry("./my_dir"))
Ok(DirEntry("./o1.txt"))
Ok(DirEntry("./o2.txt"))
Ok(DirEntry("./o3.txt"))
Ok(DirEntry("./out1.txt"))
Ok(DirEntry("./output.txt"))
Ok(DirEntry("./rust-io.ipynb"))


()

## [fs::read_link](https://doc.rust-lang.org/std/fs/fn.read_link.html)

In [29]:
fs::read_link("out2.txt")

Ok("o2.txt")

## [fs::remove_dir](https://doc.rust-lang.org/std/fs/fn.remove_dir.html)

Removes an empty directory.

In [30]:
fs::remove_dir("another")

Err(Os { code: 39, kind: Other, message: "Directory not empty" })

In [31]:
fs::remove_dir("my_dir")

Ok(())

## [fs::remove_dir_all](https://doc.rust-lang.org/std/fs/fn.remove_dir_all.html)

In [32]:
fs::remove_dir_all("another")

Ok(())

In [ ]:
https://doc.rust-lang.org/std/fs/fn.remove_file.html

In [ ]:
https://doc.rust-lang.org/std/fs/fn.rename.html


## References 

- [Trait std::io::BufRead](https://doc.rust-lang.org/std/io/trait.BufRead.html)
- [Struct std::fs::File](https://doc.rust-lang.org/std/fs/struct.File.html)
- [Struct std::io::Lines](https://doc.rust-lang.org/std/io/struct.Lines.html)
- [Linux control sequence tricks](https://www.networkworld.com/article/3284105/linux-control-sequence-tricks.html)
- [Reading from stdin: performance](https://users.rust-lang.org/t/reading-from-stdin-performance/2025)
- [What's the de-facto way of reading and writing files in Rust 1.x?](https://stackoverflow.com/questions/31192956/whats-the-de-facto-way-of-reading-and-writing-files-in-rust-1-x)
- [https://blog.yoshuawuyts.com/uninit-read-write/#summary](UNINIT READ/WRITE)
- [Rust Crate - cron](https://crates.io/crates/ron)